In [ ]:
pip install selenium

In [3]:
from selenium import webdriver
from bs4 import BeautifulSoup
from googlesearch import search
import json
import requests
from selenium.webdriver.common.by import By
import time

In [4]:
"""
This Section retrieves Youtube links from the Google search result page.
"""

batch_size = 100     #change to 1000 for 10,000 results
num_batches = 10
query = 'site:youtube.com openinapp.co'
total_results = batch_size * num_batches
results = []

for batch in range(num_batches):
    print(f"Fetching batch {batch + 1} of {num_batches}...")
    start_index = batch * batch_size
    end_index = (batch + 1) * batch_size

    for metatags in search(query, num=end_index):
        if 'youtube.com' in metatags:
            results.append(metatags)
        if len(results) >= end_index:
            break

    if len(results) >= total_results:
        break


# Save results in JSON format
with open('youtube_links.json', 'w') as file:
    json.dump(results, file, indent=4)

print(f"\nScraping completed. Retrieved {len(results)} links. Results saved in 'youtube_links.json'.")

Fetching batch 1 of 10...
Fetching batch 2 of 10...
Fetching batch 3 of 10...
Fetching batch 4 of 10...
Fetching batch 5 of 10...
Fetching batch 6 of 10...
Fetching batch 7 of 10...
Fetching batch 8 of 10...
Fetching batch 9 of 10...
Fetching batch 10 of 10...

Scraping completed. Retrieved 1000 links. Results saved in 'youtube_links.json'.


In [ ]:
%%shell

# Add debian buster
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# Prefer debian repo for chromium* packages only
# Note the double-blank lines between entries
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

In [ ]:
!apt-get update
!apt-get install chromium chromium-driver

In [7]:
def web_driver():
    options = webdriver.ChromeOptions()
    options.add_argument("--verbose")
    options.add_argument('--no-sandbox')
    options.add_argument('--headless')
    options.add_argument('--disable-gpu')
    options.add_argument("--window-size=1920, 1200")
    options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome(options=options)
    return driver
driver = web_driver()

In [11]:
links_listed = []
def scrap(class_name):
  driver.get(results[i])
  time.sleep(3)
  soup = BeautifulSoup(driver.page_source, 'html.parser')
  links = soup.find_all('a', class_ = class_name)
  for link in links:
    channel = "https://www.youtube.com"+link['href']
    return channel

for i in range(len(results)):
  if results[i][-9:] == 'community':
    results[i] = results[i][:-9]
    links_listed.append(results[i])
  elif '/post/' in results[i]:
    links_listed.append(scrap('yt-simple-endpoint style-scope ytd-backstage-post-renderer'))
  elif '/watch?' in results[i]:
    links_listed.append(scrap('yt-simple-endpoint style-scope ytd-video-owner-renderer'))
  else:
    links_listed.append(results[i])
with open('youtube_channels.json', 'w') as file:
    json.dump(links_listed, file, indent=4)

In [ ]:
driver.quit()